# 4. Fonction de hachage

In [ ]:
import math
#Note : Toutes les variables sont sur 32 bits

#Définir r comme suit : 
r = [
    7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,
    5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,
    4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,
    6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,
]
# definition de k:
k = list(range(0,64))
for i in range(0,64):
        k[i] = math.floor(abs(math.sin(i + 1)) * math.pow(2,32))

def md5(message):
    if isinstance(message, str):
        message = message.encode('utf-8')
    message = int.from_bytes(message, 'little')
    #Préparation des variables :
    
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476 

    longueur = message.bit_length()

    #Préparation du message (padding) :
    
    #ajouter le bit "1" au message
    message = (message << 1) + 1

    #ajouter le bit "0" jusqu'à ce que la taille du message en bits soit égale à 448 (mod 512)
    while message.bit_length() % 512 != 448:
        message = message << 1
        
    #ajouter la taille du message codée en 64-bit little-endian au message
    message = (message << 64) + (longueur % pow(2, 64))

    while message:
        # bloc de 512 bits
        block = message & (pow(2, 512) - 1)
        message = message >> 512

        # Division du block en 16 32-bit mots
        M = []
        while block:
            M.append(block & (pow(2, 32) - 1))
            block = block >> 32

        # Initialize hash value for this chunk
        A = h0
        B = h1
        C = h2
        D = h3

        for i in range(64):
            if i < 16:
                F = (B & C) | (bit_not(B) & D)
                g = i
            elif i < 32:
                F = (D & B) | (bit_not(D) & C)
                g = (5*i + 1) % 16
            elif i < 48:
                F = (B ^ C ^ D)
                g = (3*i + 5) % 16
            else:
                F = C ^ (B | bit_not(D))
                g = (7*i) % 16

            Temp = D
            D = C
            C = B
            B = leftrotate((A + F + k[i] + M[g]), r[i])
            A = Temp

        h0 = A
        h1 = B
        h2 = C
        h3 = D

    ret = (h0 << (32*3)) + (h1 << (32*2)) + (h2 << 32) + h3
    return ret.to_bytes(16, 'little').hex()

def bit_not(x):
    """Invert all bits in x (as opposed to ~ which will also make the number nagative)"""
    return pow(2, 32) - 1 - x

def leftrotate(x, c):
    """Rotate bits left by c bits"""
    return (x << c) & (pow(2, 32)-1) | (x >> (32-c))

In [ ]:
def inf(cle1,cle2):
    
    if((isinstance(cle1,list))):
        clef1 = cle1.copy()
        clef2 = cle2.copy()
        if ( (len(cle1) == 0) and (len(cle2) == 0)):
            return False
        c1 =  clef1.pop(0)
        c2 =  clef2.pop(0)
        if c1>c2:
            return False
        if c1<c2:
            return True
        if c1 == c2:
            return inf(clef1,clef2)
    else:
        return cle1<cle2

def eg(cle1,cle2):
    
    if(isinstance(cle1,list)):
        clef1 = cle1.copy()
        clef2 = cle2.copy()
        if ( (len(clef1) == 0) and (len(clef2) == 0)):
            return True
        c1 =  clef1.pop(0)
        c2 =  clef2.pop(0)
        if c1 == c2:
            return eg(clef1,clef2)
        else:
            return False
    else:
        return cle1 == cle2
    
def conv(cle):
    l = [] #liste des clés convertis
    tmp = [] #liste temporaire
    str1 = cle[0:8]
    tmp.append(int(str1,16))
    str2 = cle[8:16]
    tmp.append(int(str2,16))
    str3 = cle[16:24]
    tmp.append(int(str3,16))
    str4 = cle[24:32]
    tmp.append(int(str4,16))
    l.append(tmp)
    return l

def convP(oeuvre):
    l = [] #liste des clés convertis
    with open("Shakespeare/" + str(oeuvre)+".txt","r") as f:
        for line in f.readlines():
            l.append(''.join(str(ord(c)) for c in str(line)))
    return l

# 5. Arbre de Recherche

In [ ]:
import graphviz as gv
import math as m
import os as file
import matplotlib.pyplot as plt

class ArbreBinaire:
    def __init__(self, e, ssAG, ssAD):
        self.pere = None # ArbreBinaire Parent
        self.ssAG = ssAG # ArbreBinaire Gauche 
        self.ssAD = ssAD # ArbreBinaire Droit 
        self.cle = e # clé de l'ABR
        self.taille = 1
    
    def est_feuille(self):
        # Renvoi vrai si l'arbre est une feuille
        return ((self.ssAG == None) and (self.ssAD == None))
    
    def ArbreVide():
        # Renvoi un ArbreBinaire vide
        return ArbreBinaire(None, None, None)
    
    def ArbreBinaire(e, G, D):
        # cle * ArbreBinaire * ArbreBinaire -> ArbreBinaire
        # Renvoi l'Arbre Binaire dont la racine a pour contenu e, et pour fils gauche et droit (G et D)
        A = ArbreBinaire(None, None, None)
        A.cle = e
        A.ssAG = G
        A.ssAD = D
        A.ssAG.pere = A
        A.ssAD.pere = A
        return A
    
    def EstArbreVide(A):
        # ArbreBinaire -> boolean
        # Renvoi vrai ssi l'arbre A est vide
        if(A is None):
            return True
        else:
            return False
        
    def Racine(A):
        # ArbreBinaire -> cle
        # Renvoi le contenu de la racine de A
        return A.cle
    
    def SousArbreGauche(A):
        # ArbreBinaire -> ArbreBinaire
        # Renvoi une copie du sous-arbre gauche de l'arbre A
        A = A.ssAG
        return A
    
    def SousArbreDroit(A):
        # ArbreBinaire -> ArbreBinaire
        # Renvoi une copie du sous-arbre droit de l'arbre A
        A = A.ssAD
        return A
    
    def Pere(A):
        # ArbreBinaire -> ArbreBinaire
        # Renvoi l'arbre dont A est un des fils de la racine sinon vide.
        if(A.pere is None):
            return ArbreBinaire.ArbreVide()
        else:
            return A.pere
        
    def ABR_Ajout (e, A):
        # clé ∗ ArbreBinaire −> ArbreBinaire
        # Renvoie l’arbre resultant de l’ajout d'une cle dans A
        if ArbreBinaire.EstArbreVide (A):
            #print("haha")
            return ArbreBinaire (e, None, None)
        elif eg(e,A.cle):
            return A
        elif inf(e,A.cle):
            return ArbreBinaire (A.cle, 
                                 ArbreBinaire.ABR_Ajout(e, A.ssAG), A.ssAD)
        else:
            return ArbreBinaire (A.cle, 
                                 A.ssAG, ArbreBinaire.ABR_Ajout(e, A.ssAD))
        
    def ABR_recherche(e, A):
        # clé ∗ ArbreBinaire −> boolean
        # Renvoie vrai si la clé e est dans l'arbre A
        if ArbreBinaire.EstArbreVide (A):
            return False
        elif eg(e,ArbreBinaire.Racine(A)):
            return True
        elif inf(e,ArbreBinaire.Racine(A)):
            return ArbreBinaire.ABR_recherche(e,A.ssAG)
        else:
            return ArbreBinaire.ABR_recherche(e,A.ssAD)
    
        
    # Permet d'afficher l'arbre
    def plot(self):
        gtree = gv.Digraph(format='png')
        return self.to_graph(gtree,str(self.cle))
    
    def to_graph(self, g,prefixe):
        # Construit une représentation de l'arbre pour pouvoir l'afficher
        if self.est_feuille():
            g.node(prefixe,str(self.cle),shape='ellipse', **{'width':str(0.1), 'height':str(0.1)})
        else:
            g.node(prefixe,str(self.cle),shape='ellipse', **{'width':str(0.1), 'height':str(0.1)})
            if not(self.ssAG is None):
                self.ssAG.to_graph(g,prefixe+"g")
                g.edge(prefixe,prefixe+"g")
            if not(self.ssAD is None):
                self.ssAD.to_graph(g,prefixe+"d")
                g.edge(prefixe,prefixe+"d")
        return g

In [ ]:
l = convP("comedy_errors")
l_hex = []
for e in l:
    l_hex.append(md5(e))

In [ ]:
la_hex = []
for line in l_hex:
    tmp = [] #liste temporaire
    str1 = line[0:8]
    tmp.append(int(str1,16))
    str2 = line[8:16]
    tmp.append(int(str2,16))
    str3 = line[16:24]
    tmp.append(int(str3,16))
    str4 = line[24:32]
    tmp.append(int(str4,16))
    la_hex.append(tmp)

In [ ]:
A = ArbreBinaire(None,None,None)
for e in la_hex:
    if(A.cle is None):
        A.cle = e
    else:
        A = ArbreBinaire.ABR_Ajout(e, A)

## Graphique de l'abr contenant les mots hachés du fichier "comedy_errors"

In [ ]:
A.plot()

## Fonction de parcours en largeur

In [ ]:
def p_largeur(A):
    
    coucheCourant = list()
    coucheFils = list()
    l = list()
    coucheCourant.append(A)
    
    while(not(len(coucheCourant)==0)):
        
        for a in coucheCourant:
            if(a.est_feuille()):
                break
            if(ArbreBinaire.SousArbreGauche(a) != None):
                coucheFils.append(ArbreBinaire.SousArbreGauche(a))
            if(ArbreBinaire.SousArbreDroit(a) != None):
                coucheFils.append(ArbreBinaire.SousArbreDroit(a))
                
        l.append(coucheCourant[0])
        coucheCourant.pop(0)
        
        for a in coucheFils:
            if(not( a in coucheCourant)):
                coucheCourant.append(a)
        
        coucheFils = list()
    return l